In [ ]:
!pip install langchain_community google-search-results

In [ ]:
# Import necessary libraries
import os
import requests
import re
import json
import warnings
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
from serpapi import GoogleSearch

# Remove warning messages
warnings.filterwarnings("ignore")

###########################################################################################################

# Set up API keys
os.environ["OPENAI_API_KEY"] = "sk****A"  # Replace with your OpenAI API Key
google_maps_api_key = "A***U"
serpapi_key = "7***5"
geoapify_api_key = "6****d"
# Initialize OpenAI Model
llm = ChatOpenAI(model="gpt-4", temperature = 0, openai_api_key=os.getenv("OPENAI_API_KEY"))


def public_transportation(location, geoapify_api_key, radius=1000, limit=5):
    """
    Fetch transportation information near a given location using Geoapify Places API.

    Parameters:
    - location: Address or name of the location (string)
    - geoapify_api_key: Geoapify API key (string)
    - radius: Search radius in meters (default is 1000 meters)
    - limit: Maximum number of results to return (default is 10)

    Returns:
    - A list of transport hubs near the location with details, or an error message.
    """
    # Helper function to get latitude and longitude
    def lat_lng(location, geoapify_api_key):
        url = "https://api.geoapify.com/v1/geocode/search"
        params = {
            "text": location,
            "apiKey": geoapify_api_key
        }
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            if "features" in data and len(data["features"]) > 0:
                coords = data["features"][0]["geometry"]["coordinates"]
                return coords[1], coords[0]  # Return latitude, longitude
            else:
                return None, None
        except requests.exceptions.RequestException as e:
            print(f"Error fetching geolocation for '{location}': {e}")
            return None, None

    # Get latitude and longitude for the location
    lat, lng = lat_lng(location, geoapify_api_key)
    if not lat or not lng:
        return f"Could not determine the exact location for '{location}'. Please provide a valid location."

    # Use Geoapify Places API to search for transportation hubs
    url = "https://api.geoapify.com/v2/places"
    params = {
        "categories": "public_transport",  # Search for public transport hubs
        "filter": f"circle:{lng},{lat},{radius}",
        "limit": limit,
        "apiKey": geoapify_api_key
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        if "features" in data and len(data["features"]) > 0:
            transport_info = []
            for feature in data["features"]:
                name = feature["properties"].get("name", "Name not available")
                category = feature["properties"].get("categories", [])
                address = feature["properties"].get("formatted", "Address not available")
                transport_info.append(
                    f"Transport Hub: {name}\nCategories: {', '.join(category)}\nAddress: {address}\n"
                    "----------------------------------------"
                )
            return "\n\n".join(transport_info)
        else:
            return f"No transportation hubs found near '{location}'."
    except requests.exceptions.RequestException as e:
        return f"Error fetching transportation information: {e}"


tools = Tool(
    name="LocalTransport",
    func=lambda location: public_transportation(
        location, geoapify_api_key=geoapify_api_key,  # Replace with your Geoapify API key variable
    ),
    description="Fetch local transportation hubs (e.g., bus stops, train stations) near a specified location."
)

# Initialize agent with all the tools for each agent
memory = ConversationBufferMemory()
agent = initialize_agent(
    tools=[tools],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

user_query = input("Enter your request: ")
print(agent.run(user_query))